In [1]:
#Union table same tables

In [2]:
import pandas as pd

import pyodbc

import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import os

import re

In [3]:
# connection link varaibles
# NFL Data databas has all year and team in separate tables
servername = '*****'
nfl_data = 'NFL_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'

In [4]:
#create engine url to connect to sql server
engine_nfl_data = create_engine(f'mssql+pyodbc://@{servername}/{nfl_data}{trusted_conneciton}{driver}')
engine_nfl_data

Engine(mssql+pyodbc://@DESKTOP-5IAPFQC/NFL_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [5]:
# connection link varaibles
# Contains unioned tables from all years and teams
servername = '*****'
football_data = 'Football_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'

In [6]:
#create engine url to connect to sql server
engine_football_data = create_engine(f'mssql+pyodbc://@{servername}/{football_data}{trusted_conneciton}{driver}')
engine_football_data

Engine(mssql+pyodbc://@DESKTOP-5IAPFQC/Football_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [7]:
#need to fix year 2023
years = range(2010, 2024)

In [8]:
team_li = [
     '49ers',
     'Bears',
     'Bengals',
     'Broncos',
     'Browns',
     'Buccaneers',
     'Buffalos',
     'Cardinals',
     'Chargers',
     'Chiefs',
     'Colts',
     'Commanders',
     'Cowboys',
     'Dolphins',
     'Eagles',
     'Falcons',
     'Giants',
     'Jaguars',
     'Jets',
     'Lions',
     'Packers',
     'Panthers',
     'Patriots',
     'Raiders',
     'Rams',
     'Ravens',
     'Saints',
     'Seahawks',
     'Steelers',
     'Texans',
     'Titans',
     'Vikings'
]

In [9]:
table_li = [
    'team_stats_and_ranking',
    'schedule_and_game_results',
    'team_conversions',
    'passing',
    'rushing_and_receiving',
    'kick_and_punt_returns',
    'kicking',
    'punting',
    'defense_and_fumbles',
    'scoring_summary',
    'touchdown_log',
    'opponent_touchdown_log'
]

In [10]:
#Conneciton to SQL Engine
conn_nfl_data = engine_nfl_data.connect()

In [11]:
conn_football_data = engine_football_data.connect()

In [12]:
def sql_table_union (years: list, teams: list, table:str):

    sql_table_li = []
    for year in years:
        for team in team_li:
            sql_table = f'select * from {table}_{team}_{year}' 
    
            sql_table_li.append(sql_table)

    sql_table_union = ' union '.join([sqlt for sqlt in sql_table_li])
    
    return sql_table_union

In [13]:
##################### change connection engine databse 'Football_Data' when export to SQL, 
############### but not for query leave 'NFL_Data'
for table in table_li:
    print('***************************************************')
    print(f'{table}')
    sql_table_union_str = sql_table_union(years, team_li, table=table)
    
    sql_df_union = pd.read_sql(sql_table_union_str, con=conn_nfl_data)

    sql_df_union.to_sql(table, con=conn_football_data, if_exists='replace')

    sql_df_union.to_csv(f'Table Data/{table}')

***************************************************
team_stats_and_ranking
***************************************************
schedule_and_game_results
***************************************************
team_conversions
***************************************************
passing
***************************************************
rushing_and_receiving
***************************************************
kick_and_punt_returns
***************************************************
kicking
***************************************************
punting
***************************************************
defense_and_fumbles
***************************************************
scoring_summary
***************************************************
touchdown_log
***************************************************
opponent_touchdown_log


In [14]:
table = 'defense_and_fumbles'
sql_query = f'select * from {table}'

In [15]:
defense_and_fumbles_df = pd.read_sql(sql_query, con=conn_football_data)

In [16]:
defense_and_fumbles_df['Year'].value_counts()

Year
2021    1723
2022    1693
2020    1524
2023    1524
2010    1503
2018    1489
2019    1482
2015    1459
2017    1458
2014    1454
2016    1450
2011    1421
2013    1418
2012    1398
Name: count, dtype: int64

In [17]:
pd.read_csv('Table Data/opponent_touchdown_log')

,Unnamed: 0,Rank,Team,Date,Opponent,Win/Loss,Score,Quarter,Distance,Type,Detail,Year
0,0,1,49ers,09/07/2014,DAL,Win,28-17,3,2.0,rush,DeMarco Murray,2014
1,1,1,49ers,09/08/2013,GNB,Win,34-28,1,5.0,pass,Randall Cobb,2013
2,2,1,49ers,09/08/2019,TAM,Win,31-17,2,15.0,interception,Vernon Hargreaves III,2019
3,3,1,49ers,09/09/2012,GNB,Win,30-22,2,1.0,pass,Jermichael Finley,2012
4,4,1,49ers,09/09/2018,MIN,Loss,16-24,2,22.0,pass,Stefon Diggs,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
18530,18530,64,Lions,01/03/2021,MIN,Loss,35-37,1,28.0,pass,Alexander Mattison,2020
18531,18531,65,Lions,01/03/2021,MIN,Loss,35-37,2,4.0,pass,Ameer Abdullah,2020
18532,18532,66,Lions,01/03/2021,MIN,Loss,35-37,2,40.0,pass,Chad Beebe,2020
18533,18533,67,Lions,01/03/2021,MIN,Loss,35-37,3,2.0,rush,Alexander Mattison,2020
